In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
import csv

In [2]:
# TODO: need to prompt the user to input email
# USER_EMAIL = "lbo@multusmanagement.com"
# PASSWORD = "B3lieveSt0ne!"

# Account: Stephanie
# USER_EMAIL = "stephanie.francor@gmail.com"
# PASSWORD = "wDl37N16v$^a"

# Account: DSIS
# USER_EMAIL = "dsis@multusmanagement.com"
# PASSWORD = "B3lieveSt0ne!"

# Account: BHI
# USER_EMAIL = "BHI2@multusmanagement.com"
# PASSWORD = "5Az*32MjEW4p"

# Account: Alpine
# USER_EMAIL = "alpine@multusmanagement.com"
# PASSWORD = "B3lieveSt0ne!"

# Account: PT300
USER_EMAIL = "pt300@multusmanagement.com"
PASSWORD = "B3lieveSt0ne!"

MAIN_PAGE_URL = "https://www.airbnb.com/login"
LISTING_PAGE_URL = "https://www.airbnb.com/hosting/listings"
RESERVATION_URL = "https://www.airbnb.com/hosting/reservations"

XPATH_email_btn = "/html/body/div[5]/div/div/div[1]/div/div[2]/div[1]/main/div/div/div/div/div/div[3]/div/div[4]/button/div/div[2]"
XPATH_login_form = "/html/body/div[5]/div/div/div[1]/div/div[2]/div[1]/main/div/div/div/div/div/form"
XPATH_completed_btn = "/html/body/div[5]/div/div/div[1]/div/div/main/div[1]/section/div[4]/div[1]/div/div/div/div/div[1]/div/button[2]"
XPATH_canceled_btn = "/html/body/div[5]/div/div/div[1]/div/div/main/div[1]/section/div[4]/div[1]/div/div/div/div/div[1]/div/button[3]"
XPATH_reservation_next_page_btn = "/html/body/div[5]/div/div/div[1]/div/div/main/div[1]/section/footer/div/nav/div/button[8]"
CSS_SELECTOR_next_btn = "#site-content > div._1u2ubxs > section > footer > div > nav > div > button._1bfat5l.l1j9v1wn.dir.dir-ltr"
                        # "#site-content > div._1u2ubxs > section > footer > div > nav > div > button._1bfat5l.l1j9v1wn.dir.dir-ltr"
                        # "#site-content > div._1u2ubxs > section > footer > div > nav > div > button._1bfat5l.l1j9v1wn.dir.dir-ltr"


In [3]:
driver = webdriver.Chrome()
driver.get(MAIN_PAGE_URL)

In [4]:
def login(driver):
    # click continue with email with XPATH - doable
    email_btn = driver.find_element(By.XPATH, XPATH_email_btn)
    email_btn.click() 

    # try input user info - workable
    email_input = driver.find_element(By.NAME, "user[email]")
    email_input.send_keys(USER_EMAIL)

    # find "continue" button and click to submit email
    login_form = driver.find_element(By.XPATH, XPATH_login_form)
    login_form.submit()
    # TODO: after submitting, not shown (might need to sleep)
    time.sleep(5)

    # find password input form and input password
    pw_input = driver.find_element(By.NAME, "user[password]")
    pw_input.send_keys(PASSWORD)
    pw_input.submit()

    # TODO: SUCCESSFULLY LOGGED IN
    time.sleep(5)


In [5]:
login(driver)

In [6]:
driver.get(RESERVATION_URL)

In [7]:
def extract_all_entries(html_source):
    all_entries = html_source.split("<tr><td class=\"_qjegjiv\">")[1:]
    all_entries[0] = "<tr><td class=\"_qjegjiv\">" + all_entries[0]
    all_entries[-1] = all_entries[-1].split("</div></td></tr>")[0]
    return all_entries
    
def append_one_entry(df, entry):
    splitted_by_status = entry.split("<div class=\"_pgfqnw\"><span>")
    status = splitted_by_status[1].split("</span>")[0]
    splitted_by_others = splitted_by_status[1].split("<td class=\"_qjegjiv\">")
    
    check_in_time = splitted_by_others[1].split("</td>")[0]
    check_out_time = splitted_by_others[2].split("</td>")[0]
    listing_name = splitted_by_others[3].split("</td>")[1].split(">")[1]
    room = listing_name.split("*")[-1]
    confirmation_code = splitted_by_others[4].split("</td>")[0]
    total_payout = splitted_by_others[5].split("</td>")[0].split("$")[1]

    df["listing name"].append(listing_name)
    df["room"].append(room)
    df["confirmation"].append(confirmation_code)
    df["status"].append(status)
    df["check in"].append(check_in_time)
    df["check out"].append(check_out_time)
    df["total payout"].append(total_payout)
    # TODO: internal name
    
def append_one_entry_completed(df, entry):
    splitted_by_status = entry.split("<div class=\"_pgfqnw\"><span")
    status = splitted_by_status[1].split("</span>")[0][18:]
    splitted_by_others = splitted_by_status[1].split("<td class=\"_qjegjiv\">")
    
    check_in_time = splitted_by_others[1].split("</td>")[0]
    check_out_time = splitted_by_others[2].split("</td>")[0]
    listing_name = splitted_by_others[3].split("</td>")[1].split(">")[1]
    room = listing_name.split("*")[-1]
    confirmation_code = splitted_by_others[4].split("</td>")[0]
    total_payout = splitted_by_others[5].split("</td>")[0].split("$")[1]

    df["listing name"].append(listing_name)
    df["room"].append(room)
    df["confirmation"].append(confirmation_code)
    df["status"].append(status)
    df["check in"].append(check_in_time)
    df["check out"].append(check_out_time)
    df["total payout"].append(total_payout)
    # TODO: internal name
    
def append_all_entries(df, html_source):
    all_entries = extract_all_entries(html_source)
    for entry in all_entries:
        # print(entry)
        append_one_entry(df, entry)

def append_all_entries_completed(df, html_source):
    all_entries = extract_all_entries(html_source)
    for entry in all_entries:
        # print(entry)
        append_one_entry_completed(df, entry)
    
def append_all_pages_upcoming(driver, df):
    html_source = ""
    while (html_source != driver.page_source):
        html_source = driver.page_source
        append_all_entries(df, html_source)
        # next_page_btn = driver.find_element(By.XPATH, XPATH_reservation_next_page_btn)
        potential_elements = driver.find_elements(By.CSS_SELECTOR, CSS_SELECTOR_next_btn)
        if len(potential_elements) == 0:
            print("no next page")
            # break
        else:
            next_page_btn = potential_elements[0]
            next_page_btn.click()
        time.sleep(3.5)

def append_all_pages_completed(driver, df):
    html_source = ""
    # counter = 0
    while (html_source != driver.page_source):
        html_source = driver.page_source
        append_all_entries_completed(df, html_source)
        # next_page_btn = driver.find_element(By.XPATH, XPATH_reservation_next_page_btn)
        potential_elements = driver.find_elements(By.CSS_SELECTOR, CSS_SELECTOR_next_btn)
        if len(potential_elements) == 0:
            print("no next page")
            # break
        else:
            next_page_btn = potential_elements[0]
            next_page_btn.click()
        time.sleep(5)
        # counter = counter + 1

def append_all_pages_canceled_test(driver, df):
    html_source = ""
    while (html_source != driver.page_source):
        html_source = driver.page_source
        append_all_entries(df, html_source)
        # next_page_btn = driver.find_element(By.XPATH, XPATH_reservation_next_page_btn)
        potential_elements = driver.find_elements(By.CSS_SELECTOR, CSS_SELECTOR_next_btn)
        if len(potential_elements) == 0:
            break
        else:
            next_page_btn = potential_elements[0]
            next_page_btn.click()
        time.sleep(4)



In [8]:
df_dict_upcoming = {
    "listing name": [],
    "room": [],
    "confirmation": [],
    "status": [],
    "check in": [],
    "check out": [],
    "total payout": []
}
# append_all_entries_completed(df_dict_upcoming, driver.page_source)
append_all_pages_upcoming(driver, df_dict_upcoming)
df_upcoming = pd.DataFrame(df_dict_upcoming)
df_upcoming

,listing name,room,confirmation,status,check in,check out,total payout
0,Sleep6*King*Renovated*PrivateBackyard*Downtown...,L38,HMK8FQ5XHS,Currently hosting,"Feb 26, 2023","Apr 7, 2023","3,315.34"
1,Luxury*HighRise w/ PrivateBalcony*BestLocation...,22A,HMCNEA2QNH,Currently hosting,"Feb 26, 2023","Apr 22, 2023","3,953.47"
2,Sleeps8* Luxury* Backyard* NewlyRenovated* L78,L78,HMFXQEYMD2,Currently hosting,"Feb 28, 2023","Mar 31, 2023","2,578.91"
3,Luxury*Balcony w/CityView*NewlyRenovated*DTWN 20M,DTWN 20M,HM95KB8HWB,Currently hosting,"Mar 14, 2023","Apr 11, 2023","2,077.70"
4,Luxury*Balcony w/CityView*NewlyRenovated*DTWN 19M,DTWN 19M,HMZSKXM44C,Review guest,"Mar 16, 2023","Mar 20, 2023",473.71
...,...,...,...,...,...,...,...
85,Sleep 5*Balcony w/CityView*Newly Renovated* 16I,16I,HMBBYH5S8D,Confirmed,"May 5, 2023","May 8, 2023",417.70
86,Luxury* Balcony w/CityView* NewlyRenovated* 10H,10H,HMFK3QH9A9,Confirmed,"May 13, 2023","May 16, 2023",325.45
87,Jacuzzi*KingBed*BalconyCityView*NewlyRenovated...,NewlyRenovated 18C,HMSMDE9YHD,Confirmed,"May 19, 2023","May 22, 2023",455.95
88,,,HMXNNQ4RWE,Confirmed,"May 24, 2023","May 28, 2023",685.05


In [9]:
# df_upcoming.to_csv("./stephanie-reservation-upcoming.csv", index=False)
# df_upcoming.to_csv("./03-20-reservations/DSIS-reservation-upcoming.csv", index=False)
# df_upcoming.to_csv("./BHI-reservation-upcoming.csv", index=False)
# df_upcoming.to_csv("./03-20-reservations/LBO-reservation-upcoming.csv", index=False)
df_upcoming.to_csv("./03-20-reservations/PT300-reservation-upcoming.csv", index=False)

In [101]:
# df.to_csv("./out.csv", index=False)
# df["xxxx"] == "arriving ......"

In [10]:
# getting completed reservations
completed_bnt = driver.find_element(By.XPATH, XPATH_completed_btn)
completed_bnt.click()

In [11]:
df_dict_completed = {
    "listing name": [],
    "room": [],
    "confirmation": [],
    "status": [],
    "check in": [],
    "check out": [],
    "total payout": []
}

append_all_pages_completed(driver, df_dict_completed)
df_completed = pd.DataFrame(df_dict_completed)
df_completed

,listing name,room,confirmation,status,check in,check out,total payout
0,Luxury* Balcony w/CityView* NewlyRenovated* 6N,6N,HMHTQ4XZNS,Review guest,"Mar 17, 2023","Mar 19, 2023",295.95
1,Sleep 5* Luxury* Balcony *NewlyRenovated* 15J,15J,HMYZHFBZND,Review guest,"Mar 17, 2023","Mar 19, 2023",272.91
2,Luxury*PrivateBalcony*Renovated*PrimeLocation*2M,2M,HMBYTZNXQT,Review guest,"Mar 11, 2023","Mar 19, 2023",813.51
3,Luxury* Balcony w/CityView* NewlyRenovated* 15K,15K,HMFSHZR9DC,Review guest,"Mar 16, 2023","Mar 19, 2023",415.44
4,*Luxury *Balconyw/CityView* Renovated* Downtow...,Downtown 8H,HM3SSE2224,Review guest,"Mar 16, 2023","Mar 19, 2023",392.89
...,...,...,...,...,...,...,...
1707,*Luxury *Balconyw/CityView* Renovated* Downtow...,Downtown 8H,HMNDRRFT8C,Past guest,"May 15, 2021","May 17, 2021",152.10
1708,*Luxury*Balcony w/CityView*NewlyRenovated* 10M,10M,HMJHYZNB8C,Past guest,"May 13, 2021","May 15, 2021",239.59
1709,*Luxury *Balconyw/CityView* Renovated* Downtow...,Downtown 8H,HMXD9CNHHF,Past guest,"May 13, 2021","May 15, 2021",209.52
1710,Sleep 5* Renovated* CityView* Spacious*DTWN* 23J,23J,HMMPKYZSEJ,Past guest,"May 13, 2021","May 15, 2021",236.10


In [12]:
# df_completed.to_csv("./stephanie-reservation-completed.csv", index=False)
# df_completed.to_csv("./03-20-reservations/DSIS-reservation-completed.csv", index=False)
# df_completed.to_csv("./BHI-reservation-completed.csv", index=False)
# df_completed.to_csv("./03-20-reservations/LBO-reservation-completed.csv", index=False)
df_completed.to_csv("./03-20-reservations/PT300-reservation-completed.csv", index=False)

In [13]:
# getting canceled reservations
canceled_bnt = driver.find_element(By.XPATH, XPATH_canceled_btn)
canceled_bnt.click()

In [14]:
df_dict_canceled = {
    "listing name": [],
    "room": [],
    "confirmation": [],
    "status": [],
    "check in": [],
    "check out": [],
    "total payout": []
}

append_all_pages_canceled_test(driver, df_dict_canceled)
df_canceled = pd.DataFrame(df_dict_canceled)
df_canceled

,listing name,room,confirmation,status,check in,check out,total payout
0,Discard* Balcony w/CityView* NewlyRenovated* 10H,10H,HMYX5BD4TM,Canceled by guest,"May 4, 2023","May 8, 2023",0.00
1,Downtown High Story Condo with Patio and Amazi...,Downtown High Story Condo with Patio and Amazi...,HMWQXAWWFR,Canceled by guest,"Apr 29, 2023","May 1, 2023",0.00
2,Discard* Balcony w/CityView* NewlyRenovated* 10H,10H,HM9N2A4S8J,Canceled by guest,"Apr 28, 2023","May 1, 2023",0.00
3,Sleeps5*Luxury*HighFloor*Balcony *Renovated 15I,Renovated 15I,HMH2W2SNCE,Canceled by guest,"Apr 28, 2023","Apr 30, 2023",31.33
4,Sleeps5*Luxury*HighFloor*Balcony *Renovated 15I,Renovated 15I,HM5ZZTTSK2,Canceled by guest,"Apr 28, 2023","Apr 30, 2023",0.00
...,...,...,...,...,...,...,...
392,NEW Central Location 1 BR l Downtown ATL with ...,NEW Central Location 1 BR l Downtown ATL with ...,HMF9SF25PT,Canceled by guest,"Jun 11, 2021","Jun 13, 2021",189.63
393,Discard: 300PT 16I,Discard: 300PT 16I,HMY43NBBZX,Canceled by guest,"Jun 10, 2021","Jun 18, 2021",0.00
394,*Luxury *Balconyw/CityView* Renovated* Downtow...,Downtown 8H,HMKAZSK9TK,Canceled by guest,"Jun 8, 2021","Jun 10, 2021",0.00
395,HighFloor(22FL) l Bright and Stylish 1BR in Do...,HighFloor(22FL) l Bright and Stylish 1BR in Do...,HMTBH3AT2X,Canceled by guest,"May 28, 2021","May 31, 2021",0.00


In [15]:
# df_canceled.to_csv("./stephanie-reservation-canceled.csv", index=False)
# df_canceled.to_csv("./03-20-reservations/DSIS-reservation-canceled.csv", index=False)
# df_canceled.to_csv("./BHI-reservation-canceled.csv", index=False)
# df_canceled.to_csv("./03-20-reservations/LBO-reservation-canceled.csv", index=False)
df_canceled.to_csv("./03-20-reservations/PT300-reservation-canceled.csv", index=False)

In [ ]:
# internal name XPATH
# /html/body/div[6]/div/div/div[1]/div/div/main/div/div/div/div/section[1]/div/div/div[2]/div[1]/table/tbody[2]/tr[1]/td[2]/div/div[2]/div[2]